# sim plan

* for every round
  * procedure?
        * if yes: 
            * offer to random resident (and choose intern who would have gotten it either way for log)
                * if accepts: increment and log to both
                * else: log miss only
        * else
            * log missed procedure0
    * 

In [ ]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from icecream import ic
from statsmodels.distributions.discrete import poisson

In [ ]:
# custom datatype for procedure offer logs
Procedure_Outcome = namedtuple(
    "Procedure_Outcome",
    [
        "bootstrap_run",
        "day",
        "senior",
        "senior_prior_procedures",
        "intern",
        "intern_prior_procedures",
        "nth_procedure_opportunity_of_the_day",
        "performed",
    ],
)

In [ ]:
residents = pd.read_csv("residents.csv")
original_resident_data = residents.copy()  # copy for resetting between bootstrap runs
residents

## demonstration of Poisson distribution to generate random procedure offers

### TODO: generate different starting conditions programatically

In [ ]:
poisson_lambda = 1 / 7
p = poisson(poisson_lambda)
plt.bar(np.arange(0, 10), p.pmf(np.arange(0, 10)))

for example, a month's worth with $\lambda$ = 1/14, which is a Q2wk average divided out

In [ ]:
random_year = p.rvs(365)
random_year

In [ ]:
random_year.sum() / 365 - poisson_lambda

pretty close, and gets the randomness right

### TODO: bootstrapping

In [ ]:
days = 365
signoff_threshold = 5
signed_off_avidity = 0.5
not_signed_off_avidity = 0.1
bootstrap_runs = 10

seniors = residents.training_level == "senior"
interns = residents.training_level == "intern"

R2s = 10
R3s = 10
R1s = 10

In [ ]:
residents

In [ ]:
outcome_log = list()
bootstrap_log = list()

for bootstrap_run in range(bootstrap_runs):

    # reset residents to initial conditions
    residents = original_resident_data.copy()

    for procedures_today, day in zip(p.rvs(days), range(days)):

        for procedure_opportunity in range(procedures_today):

            selected_senior = residents[seniors].sample().name.values[0]
            selected_intern = residents[interns].sample().name.values[0]

            if (
                residents.loc[
                    residents.name.isin([selected_senior, selected_intern]),
                    "paracentesis_completed",
                ]
                >= signoff_threshold
            ).any():
                avidity = signed_off_avidity
            else:
                avidity = not_signed_off_avidity

            senior_prior_procedures = residents.loc[
                residents.name == selected_senior, "paracentesis_completed"
            ].values[0]
            intern_prior_procedures = residents.loc[
                residents.name == selected_intern, "paracentesis_completed"
            ].values[0]

            performed = None

            if np.random.rand() <= avidity:

                residents.loc[
                    residents.name == selected_senior, "paracentesis_completed"
                ] += 1

                residents.loc[
                    residents.name == selected_intern, "paracentesis_completed"
                ] += 1
                performed = True
            else:
                performed = False

            today_outcome = Procedure_Outcome(
                bootstrap_run=bootstrap_run,
                day=day,
                senior=selected_senior,
                senior_prior_procedures=senior_prior_procedures,
                intern=selected_intern,
                intern_prior_procedures=intern_prior_procedures,
                nth_procedure_opportunity_of_the_day=procedure_opportunity,
                performed=performed,
            )
            outcome_log.append(today_outcome)
    outcomes = pd.DataFrame(outcome_log)
    bootstrap_log.append(outcomes)

# TODO: good place to convert to polars for analytics if slow
bootstrapped_outcomes = pd.concat(bootstrap_log)

In [ ]:
bootstrapped_outcomes

In [ ]:
bootstrapped_outcomes.groupby("bootstrap_run").performed.sum()

In [ ]:
# average
bootstrapped_outcomes.groupby("bootstrap_run").performed.sum().sum() / bootstrap_runs

### TODO rewrite analytics with groupbys

In [ ]:
outcomes.performed.sum()

In [ ]:
# proportion of available procedures actually done
outcomes.performed.sum() / len(outcomes)

In [ ]:
# seniors signed off this block
outcomes.loc[
    (outcomes.senior_prior_procedures == (signoff_threshold - 1)) & (outcomes.performed)
]

In [ ]:
# interns signed off this block
outcomes.loc[
    (outcomes.intern_prior_procedures == (signoff_threshold - 1)) & (outcomes.performed)
]

In [ ]:
# residents signed off
residents.loc[residents.paracentesis_completed >= signoff_threshold]

In [ ]:
# residents not signed off
residents.loc[residents.paracentesis_completed < signoff_threshold]